In [7]:
spark.stop()
#!pip install findspark

In [74]:
## This is for correct the file-format, because some chinese sign cann't be read by utf-8
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import col, array_contains, split, when, regexp_replace

# # creat SparkSession
# spark = SparkSession.builder \
#     .appName("ETL_media") \
#     .getOrCreate()
# #file_path = r'F:\申请，润\我去澳洲\university of adelaide\AWS\video_data.csv'
# # load CSV 
# # File layout - inferSchema 
# # Field delimiter – pipe, tab, comma
# # File encoding – ASCII, UTF-8, UTF-16,gb2312 中文字符
# # API - read.csv
# #df = spark.read.csv(file_path, header=True, inferSchema=True,encoding="gb2312")
# df = spark.read.csv("video_data.csv", header=True, inferSchema=True,encoding="gb2312") # automately recognize the type of data
# df = df.drop("_c3")
# df.show(1,truncate=False)

# df = df.withColumn("VideoTitle", regexp_replace(col("VideoTitle"), "’", "'")) \
#        .withColumn("VideoTitle", regexp_replace(col("VideoTitle"), "‘", "'")) \
#         .withColumn("VideoTitle", regexp_replace(col("VideoTitle"), "”", "\"")) \
#         .withColumn("VideoTitle", regexp_replace(col("VideoTitle"), "“", "\""))
# df.show(200,truncate=False)
# file_name1 = r"F:\申请，润\我去澳洲\university of adelaide\AWS\ETL\source"
# df.coalesce(1).write.csv(file_name1, mode='overwrite', header=True, encoding='UTF-8')

In [1]:
import findspark 

findspark.init()
findspark.find()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, array_contains, split, when, regexp_replace

# creat SparkSession
spark = SparkSession.builder \
    .appName("ETL_media") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.cores", "4") \
    .config("spark.driver.cores", "1") \
    .getOrCreate()
print(f"Spark Executor Memory: {spark.conf.get('spark.executor.memory')}")
print(f"Spark Driver Memory: {spark.conf.get('spark.driver.memory')}")
print(f"Spark Executor Cores: {spark.conf.get('spark.executor.cores')}")
print(f"Spark Driver Cores: {spark.conf.get('spark.driver.cores')}")

#file_path = r'F:\申请，润\我去澳洲\university of adelaide\AWS\video_data.csv'
# load CSV 
# File layout - inferSchema 
# Field delimiter – pipe, tab, comma
# File encoding – ASCII, UTF-8, UTF-16,gb2312 中文字符
# API - read.csv
#df = spark.read.csv(file_path, header=True, inferSchema=True,encoding="gb2312")
df = spark.read.csv("video_data_new.csv", header=True, inferSchema=True,encoding="UTF-8") # automately recognize the type of data
df.show(1,truncate=False)



Spark Executor Memory: 8g
Spark Driver Memory: 4g
Spark Executor Cores: 4
Spark Driver Cores: 1
+-------------------+---------------------------------------------------------+-----------------------------------------------+
|DateTime           |VideoTitle                                               |events                                         |
+-------------------+---------------------------------------------------------+-----------------------------------------------+
|2024-01-11 10:30:31|App Web|Clips|a-current-affair;2016|William Tyrrell twist|157,120,160,104,162,000,000,000,000,000,000,000|
+-------------------+---------------------------------------------------------+-----------------------------------------------+
only showing top 1 row



In [2]:
## set some wrong record for testing

col_name = "DateTime"
target_value = "2024-01-11 10:30:31"
target_value1 = "2024-01-11 10:33:51"

# 使用 withColumn 和 when 函数来条件性地更新特定单元格的值
df = df.withColumn(col_name,
                   when(col(col_name) == target_value, None).otherwise(col(col_name)))
df = df.withColumn(col_name,
                   when(col(col_name) == target_value1, "202401#1110k33l51").otherwise(col(col_name)))

# 显示更新后的数据
print("DataFrame with modified cell:")
df.show(10,truncate=False)
df.columns
df.count()
df.printSchema()



DataFrame with modified cell:
+-------------------+-----------------------------------------------------------------------+---------------------------------------------------+
|DateTime           |VideoTitle                                                             |events                                             |
+-------------------+-----------------------------------------------------------------------+---------------------------------------------------+
|NULL               |App Web|Clips|a-current-affair;2016|William Tyrrell twist              |157,120,160,104,162,000,000,000,000,000,000,000    |
|2024-01-11 10:30:53|news| Shark attacks spearfisherman                                     |127,157,120,160,104,000,000,000,000                |
|2024-01-11 10:30:21|news| Shark attacks spearfisherman                                     |127,157,120,160,104,000,000,000,000,000,000,000,000|
|2024-01-11 10:31:27|news| Chilean navy films UFO                                           |1

# Json

In [3]:
# 读取JSON文件
df_user = spark.read.json("user.json")
df_user.show(1,truncate =False)
df_user.printSchema()
df_user.count()


+------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id                                  |profile                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
+------------------------------------+

190000

In [4]:
from pyspark.sql.functions import broadcast
df_user_id = df_user.na.drop().select("id").distinct()

df_user_id.show(10)
df_user_id.count()

# Broadcast the user IDs
# broadcast_user_ids = spark.sparkContext.broadcast(df_user_id.collect())

# 在PySpark中，使用广播变量（Broadcast Variables）时，
# 需要将数据从分布式的Executor节点传输到驱动程序节点（Driver节点）。
# 这是因为广播变量是在驱动程序上创建和管理的，然后将其分发到工作节点上的任务中使用。
# 具体来说，为了在广播变量中使用DataFrame或RDD的数据，需要使用collect()方法将数据收集到驱动程序节点。

+--------------------+
|                  id|
+--------------------+
|557c2960-c218-4ee...|
|ef4260d6-500d-4ec...|
|5d7c0aeb-515b-4d2...|
|c37de051-44e8-429...|
|58abded2-f45f-4f9...|
|ec497bd4-ac2b-4a0...|
|436d7423-265a-49e...|
|feff51e1-149c-4ef...|
|116cc000-a458-4cf...|
|824f8f2e-1512-4b8...|
+--------------------+
only showing top 10 rows



190000

In [5]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import pandas as pd
import random

# 假设 id_df 是一个包含 ID 的 DataFrame，video_plays_df 是视频播放信息的 DataFrame

# 将 id_df 转换为 Pandas DataFrame，并随机选择其中的 ID
id_pandas_df = df_user_id.toPandas()
random_ids = id_pandas_df["id"].tolist()
random.shuffle(random_ids)

# 定义一个函数来随机选择 ID
def assign_random_id():
    return random.choice(random_ids)


# 注册 UDF
assign_random_id_udf = udf(assign_random_id, StringType())

# # 给 video_plays_df 添加一个名为 "id" 的列，值为随机选择的 ID
df = df.withColumn("id", assign_random_id_udf())

# # 显示添加了 ID 列后的 DataFrame
df.show(100)

C:\Users\nining\AppData\Local\Temp\ipykernel_7796\4165628295.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


+-------------------+--------------------+--------------------+--------------------+
|           DateTime|          VideoTitle|              events|                  id|
+-------------------+--------------------+--------------------+--------------------+
|               NULL|App Web|Clips|a-c...|157,120,160,104,1...|bba4a92f-32af-480...|
|2024-01-11 10:30:53|news| Shark attac...|127,157,120,160,1...|47c0d0ce-2641-4ce...|
|2024-01-11 10:30:21|news| Shark attac...|127,157,120,160,1...|25b25121-cd88-4cc...|
|2024-01-11 10:31:27|news| Chilean nav...|157,120,160,104,1...|2539be2f-e94c-4f2...|
|2024-01-11 10:30:33|news| Video shows...|157,120,160,104,1...|ebef8dd1-e411-4f3...|
|2024-01-11 10:30:18|App Web|Clips|tod...|127,157,120,160,1...|9d3394e9-8222-429...|
|2024-01-11 10:30:22|news| Australian ...|127,157,120,160,1...|f4ae98b8-db29-4d2...|
|2024-01-11 10:32:50|news| Chilean nav...|127,157,120,160,1...|3177bc7b-2f5a-444...|
|2024-01-11 10:32:37|news| Australian ...|157,120,160,104,1...|95

In [6]:
file_name = rf"F:\申请，润\我去澳洲\university of adelaide\AWS\ETL\data_source_id"
#异常值写入csv日志

df.coalesce(1).write.csv(file_name, header=True)
